In [1]:
import uproot
import numpy as np
import awkward as ak

import sys
sys.path.append("..")
from hists.parameters import synchrotronBeamEnergiesMap

In [2]:
tree = uproot.open('/data_cms_upgrade/cuisset/testbeam18/clue3d/v38/cmssw/data/CLUE_clusters.root:clusters')

In [3]:
def findEventsWithLowRecoEnergy(energyFractionThreshold = 0.8, stopAfter=100):
    """ Find events where the total reconstructed energy is less than *energyFractionThreshold* of the beamEnergy (not incl synch losses)
    """
    filtered_arrays = []
    count = 0
    for array in tree.iterate(step_size="100MB", library="ak",
            filter_name=["event", "ntupleNumber", "beamEnergy", "rechits_energy"]):
        cut = ak.sum(array.rechits_energy, axis=-1)<array.beamEnergy*energyFractionThreshold
        filtered_arrays.append(ak.copy(array[cut]))
        count += np.sum(cut)
        if count > stopAfter:
            break
    return ak.concatenate(filtered_arrays)

In [11]:
findEventsWithLowRecoEnergy()[["beamEnergy", "ntupleNumber", "event"]]

<Array [{beamEnergy: 300, ...}, ..., {...}] type='336 * {beamEnergy: float3...'>

In [6]:
def printForSpreadsheet(ar:ak.Array):
    for event in ar:
        print(";".join([str(int(i)) for i in [event.beamEnergy, event.ntupleNumber, event.event]]))

In [7]:
def findEventsWithNoTrackster():
    pass # TODO

In [8]:
def findEventsWithCLUE3DIssue(thresholdClue3D = 0.8, lowerThresholdRechits=0.9, stopAfter=100):
    """ Find events where CLUE3D reconstructs less than thresholdClue3D*beamEnergy whereas at rechits level at least lowerThresholdRechits*beamEnergy is reconstructed.
    Beam energy includes synchrotron losses
    """
    filtered_arrays = []
    count = 0
    energyMap = np.vectorize(synchrotronBeamEnergiesMap.get, otypes=[float])
    for array in tree.iterate(step_size="100MB", library="ak", #cut="beamEnergy==100", 
            filter_name=["event", "ntupleNumber", "beamEnergy", "rechits_energy", "clus3D_energy"]):
        if len(array) == 0:
            continue
        synchBeamEnergies = energyMap(array.beamEnergy)

        # fill_none is required to drop events which have no 3D clusters
        cut = ak.fill_none((ak.max(array.clus3D_energy, axis=-1) < thresholdClue3D * synchBeamEnergies) &
         (ak.sum(array.rechits_energy, axis=-1) > lowerThresholdRechits * synchBeamEnergies),
         value=False
        )
        
        cut_array = array[cut]
        filtered_arrays.append(ak.copy(cut_array))
        count += np.sum(cut)
        if count > stopAfter:
            break
    return ak.concatenate(filtered_arrays)

In [9]:
ar = findEventsWithCLUE3DIssue(0.5, stopAfter=1000000000000000)
ar

<Array [{beamEnergy: 300, ...}, ..., {...}] type='326 * {beamEnergy: float3...'>

In [10]:
printForSpreadsheet(ar)

300;435;5155
20;436;575
20;436;1326
20;436;1848
20;436;2427
20;436;2610
20;437;780
20;437;1956
20;437;3024
20;437;3154
20;437;3534
20;437;4238
20;437;6341
20;437;7039
20;437;7110
20;437;9772
20;437;10160
20;439;892
20;439;3579
20;439;4318
20;439;4385
20;439;4867
20;439;5477
20;439;6059
20;439;6294
20;439;8663
20;439;9500
20;439;10064
20;441;38
20;441;297
20;441;829
20;441;1032
20;441;1703
20;441;1740
20;441;2072
20;441;2812
20;441;4066
20;441;5222
20;441;5726
20;441;5943
20;441;6425
20;441;8986
20;441;9905
20;442;225
20;442;722
20;442;1997
20;442;2662
20;442;2828
20;442;5876
20;442;9449
20;442;10227
20;443;1439
20;443;1831
20;443;2097
20;444;556
20;444;759
20;444;1144
20;444;2159
20;444;2437
20;444;2484
20;444;3088
20;444;4233
20;444;4249
20;444;4435
20;444;5075
20;444;5271
20;444;5342
20;444;6484
20;444;7036
20;444;7085
20;444;7998
20;444;8551
20;444;9060
20;444;9726
20;447;943
20;447;1623
20;447;3133
20;447;5985
20;447;6194
20;450;4744
20;450;4878
20;450;5850
20;450;6610
20;450;7965
